# AIM

ニューラルネットワークで排他的論理和回路のモデル ver. softmax with TensorFlow

In [12]:
import time
import numpy as np
import tensorflow as tf

In [2]:
from pkg_resources import get_distribution
import platform
print("python", platform.python_version())
print("")
libs = ["numpy", "tensorflow"]
for lib in libs:
    version = get_distribution(lib).version
    print(lib, version)

python 3.5.2

numpy 1.13.1
tensorflow 1.3.0


In [138]:
# ネットワークの定義

in_size = 2
hidden_size = 5
out_size = 2
# プレースホルダー
x_ = tf.placeholder(tf.float32, shape=[None, in_size])
y_ = tf.placeholder(tf.float32, shape=[None, out_size])
# 順伝播のネットワークを作成
fc1_w = tf.Variable(tf.truncated_normal([in_size, hidden_size], stddev=0.1), dtype=tf.float32) # 入力層の重み
fc1_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32) # 入力層のバイアス
fc1 = tf.nn.sigmoid(tf.matmul(x_, fc1_w) + fc1_b) # 全結合
fc2_w = tf.Variable(tf.truncated_normal([hidden_size, hidden_size], stddev=0.1), dtype=tf.float32) # 隠れ層の重み
fc2_b = tf.Variable(tf.constant(0.1, shape=[hidden_size]), dtype=tf.float32) # 隠れ層のバイアス
fc2 = tf.nn.sigmoid(tf.matmul(fc1, fc2_w) + fc2_b) # 全結合
fc3_w = tf.Variable(tf.truncated_normal([hidden_size, out_size], stddev=0.1), dtype=tf.float32) # 出力層の重み
fc3_b = tf.Variable(tf.constant(0.1, shape=[out_size]), dtype=tf.float32) # 出力層のバイアス
y_pre = tf.nn.sigmoid(tf.matmul(fc2, fc3_w) + fc3_b) # 全結合
# クロスエントロピー誤差
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_pre))
# 勾配法
train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)
# 正解率の計算
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [146]:
# 学習

EPOCH_NUM = 2000
BATCH_SIZE = 4

# 教師データ
train_data = [
    [[0, 0], [0]],
    [[1, 0], [1]],
    [[0, 1], [1]],
    [[1, 1], [0]]
]

# 教師データを変換
N = len(train_data) # 教師データの総数
train_x, train_y = [], []
for x, y in train_data:
    train_x.append(x)
    train_y.append(np.eye(2)[y][0])
train_x = np.array(train_x, dtype="float32")
train_y = np.array(train_y, dtype="float32")

# 学習
print("Train")
with tf.Session() as sess:
    st = time.time()
    sess.run(tf.global_variables_initializer())
    for epoch in range(EPOCH_NUM):
        perm = np.random.permutation(N)
        total_loss = 0
        total_accuracy = 0
        for i in range(0, N, BATCH_SIZE):
            batch_x = train_x[perm[i:i+BATCH_SIZE]]
            batch_y = train_y[perm[i:i+BATCH_SIZE]]
            total_loss += cross_entropy.eval(feed_dict={x_: batch_x, y_: batch_y})
            train_step.run(feed_dict={x_: batch_x, y_: batch_y})
            total_accuracy += accuracy.eval(feed_dict={x_: batch_x, y_: batch_y})
        mean_accuracy = total_accuracy/(N/BATCH_SIZE)
        if (epoch+1) % 500 == 0:
            ed = time.time()
            print("epoch:\t{}\ttotal loss:\t{}\tmean accuracy:\t{}\ttime:\t{}".format(epoch+1, total_loss, mean_accuracy, ed-st))
            st = time.time()
            
    # 予測
    print("\nPredict")
    def predict(x):
        print(x, "=>", np.argmax(y_pre.eval(feed_dict={x_: x})))
    predict([[1, 0]])
    predict([[0, 0]])
    predict([[1, 1]])
    predict([[0, 1]])


Train
epoch:	500	total loss:	0.31562888622283936	mean accuracy:	1.0	time:	3.308142900466919
epoch:	1000	total loss:	0.3138287365436554	mean accuracy:	1.0	time:	0.9018409252166748
epoch:	1500	total loss:	0.3135184645652771	mean accuracy:	1.0	time:	0.9040908813476562
epoch:	2000	total loss:	0.31340599060058594	mean accuracy:	1.0	time:	0.8946030139923096

Predict
[[1, 0]] => 1
[[0, 0]] => 0
[[1, 1]] => 0
[[0, 1]] => 1
